In [16]:
# ==============================================================================
# 1. SETUP - INSTALL AND IMPORT LIBS
# ==============================================================================
!pip install langgraph langchain_core==0.1.52 --quiet

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import TypedDict
import time # Import time to generate a timestamp

# ==============================================================================
# 2. DEFINE THE STATE AND WORKFLOW LOGIC
# ==============================================================================
class ApprovalState(TypedDict, total=False):
    item: str
    auto_summary: str
    human_decision: str
    final_status: str

def generate_summary(state: ApprovalState) -> dict:
    print(f"🤖 AI is generating a summary for: {state['item']}...")
    summary = f"This is an AI-generated summary for '{state['item']}'. It appears to be a high-priority item requiring review."
    return {"auto_summary": summary}

def process_decision(state: ApprovalState) -> dict:
    print("\n--- Workflow Resumed ---")
    decision = state.get("human_decision")
    print(f"👤 Human decision received: '{decision}'")
    status = "APPROVED" if decision == "approve" else "REJECTED"
    return {"final_status": status}

# Dummy pass-through node for human approval pause
def human_approval_node(state: ApprovalState) -> dict:
    # This node doesn't modify state, it just acts as a pause point
    return state

# ==============================================================================
# 3. BUILD AND COMPILE THE WORKFLOW
# ==============================================================================
def create_approval_workflow():
    workflow = StateGraph(ApprovalState)

    workflow.add_node("generate_summary", generate_summary)
    workflow.add_node("human_approval_node", human_approval_node)
    workflow.add_node("process_decision", process_decision)

    workflow.add_edge(START, "generate_summary")
    workflow.add_edge("generate_summary", "human_approval_node")
    workflow.add_edge("human_approval_node", "process_decision")
    workflow.add_edge("process_decision", END)

    memory = SqliteSaver.from_conn_string(":memory:")

    # Human-in-the-loop: workflow pauses before "human_approval_node"
    return workflow.compile(
        checkpointer=memory,
        interrupt_before=["human_approval_node"]
    )

# ==============================================================================
# 4. RUN THE WORKFLOW AND DEMONSTRATE THE PAUSE/RESUME CYCLE
# ==============================================================================
if __name__ == "__main__":
    app = create_approval_workflow()

    # Generate a timestamp for thread_ts
    thread_ts = str(int(time.time()))
    # Corrected config: Nest thread_id and thread_ts under 'configurable'
    thread_config = {"configurable": {"thread_id": "approval-456", "thread_ts": thread_ts}}
    initial_state = {"item": "Project Phoenix Launch Plan"}

    print("--- Part 1: Running workflow until human approval is needed ---")
    interrupt_state = app.invoke(initial_state, config=thread_config)

    print("\n--- Workflow Paused: Awaiting Human Input ---")
    print("State at pause:")
    for k, v in interrupt_state.items():
        print(f"  {k}: {v}")

    print("\n--- Part 2: Providing human input to resume the workflow ---")
    # Need to provide the same thread_id and thread_ts to resume
    # Corrected config for resume: Nest thread_id and thread_ts under 'configurable'
    final_result = app.invoke({"human_decision": "approve"}, config=thread_config)

    print("\n✅ Final Workflow Status:")
    for k, v in final_result.items():
        print(f"  {k}: {v}")

--- Part 1: Running workflow until human approval is needed ---
🤖 AI is generating a summary for: Project Phoenix Launch Plan...

--- Workflow Paused: Awaiting Human Input ---
State at pause:
  item: Project Phoenix Launch Plan
  auto_summary: This is an AI-generated summary for 'Project Phoenix Launch Plan'. It appears to be a high-priority item requiring review.

--- Part 2: Providing human input to resume the workflow ---
🤖 AI is generating a summary for: None...

✅ Final Workflow Status:
  auto_summary: This is an AI-generated summary for 'None'. It appears to be a high-priority item requiring review.
  human_decision: approve
